In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from glob import glob
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
import math
from sklearn.preprocessing import LabelEncoder

RANDOM_SEED = 42
NUM_CLASSES = 5


"""
buy_post - целевая переменная
date_diff_post - целевая переменная
"""

'\nbuy_post - целевая переменная\ndate_diff_post - целевая переменная\n'

In [35]:
df_train = pd.read_csv('main_train.csv')

In [27]:
df_train

,Unnamed: 0,index,customer_id,buy_post,date_diff_post,popular_dish_product,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval,popular_dish
0,0,0,29891,1,9.0,0,149.627941,439.029412,6,589,625,300.00,300.00,389.97,119.99,0.100428,0,25,2.316445,47.0
1,1,1,30477,1,10.0,0,93.042623,320.000000,2,632,461,320.00,320.00,99.99,399.99,0.192454,0,25,2.079109,588.0
2,2,2,31426,1,4.0,0,109.227791,153.000000,7,786,857,153.00,153.00,49.99,99.98,0.122477,0,24,2.091698,900.0
3,3,3,44491,1,42.0,0,51.490000,128.884000,3,510,858,139.00,139.00,344.97,49.97,0.012164,0,4,6.960298,857.0
4,4,4,44939,1,9.0,0,66.593200,171.474400,7,899,900,129.27,129.27,604.93,554.94,7.181852,0,3,5.149244,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499995,499995,46624509,0,NaN,0,85.081818,112.530000,7,978,720,112.53,112.53,514.95,99.99,0.014687,0,3,0.407315,978.0
499996,499996,499996,46625265,0,NaN,0,92.297692,120.000000,8,987,900,120.00,120.00,349.96,49.99,0.008623,0,3,0.005027,900.0
499997,499997,499997,46639170,1,10.0,0,99.083636,80.200000,7,847,847,80.20,80.20,169.99,399.97,0.048738,0,3,0.047388,847.0
499998,499998,499998,46654016,1,15.0,0,203.321667,270.000000,6,656,652,270.00,270.00,469.96,179.99,0.076979,0,3,0.083773,656.0


In [36]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier

In [37]:
X, y = df_train.drop(columns=['customer_id', 'index', 'date_diff_post']), df_train[['date_diff_post', 'buy_post']]

In [38]:
X = X.drop(columns=['Unnamed: 0','popular_dish_product'])
X = X[['popular_dish','avg_revenue','avg_sqm','popular_format','m1_dish','m2_dish','m1_sqm','m2_sqm','m1_revenue','m2_revenue','m2_interval','mean_delta_time','visits','mean_interval','buy_post']]
X

,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval,buy_post
0,47.0,149.627941,439.029412,6,589,625,300.00,300.00,389.97,119.99,0.100428,0,25,2.316445,1
1,588.0,93.042623,320.000000,2,632,461,320.00,320.00,99.99,399.99,0.192454,0,25,2.079109,1
2,900.0,109.227791,153.000000,7,786,857,153.00,153.00,49.99,99.98,0.122477,0,24,2.091698,1
3,857.0,51.490000,128.884000,3,510,858,139.00,139.00,344.97,49.97,0.012164,0,4,6.960298,1
4,87.0,66.593200,171.474400,7,899,900,129.27,129.27,604.93,554.94,7.181852,0,3,5.149244,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,978.0,85.081818,112.530000,7,978,720,112.53,112.53,514.95,99.99,0.014687,0,3,0.407315,0
499996,900.0,92.297692,120.000000,8,987,900,120.00,120.00,349.96,49.99,0.008623,0,3,0.005027,0
499997,847.0,99.083636,80.200000,7,847,847,80.20,80.20,169.99,399.97,0.048738,0,3,0.047388,1
499998,656.0,203.321667,270.000000,6,656,652,270.00,270.00,469.96,179.99,0.076979,0,3,0.083773,1


In [39]:
X = X.fillna(-1)
y = y.fillna(-1)

Предсказываем вернется или нет

Начинаем регрессию

In [41]:
X = X.drop(columns = ['date_diff_post'])

In [42]:
X = X[X['buy_post']==1]
X

,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval,buy_post
0,47.0,149.627941,439.029412,6,589,625,300.00,300.00,389.97,119.99,0.100428,0,25,2.316445,1
1,588.0,93.042623,320.000000,2,632,461,320.00,320.00,99.99,399.99,0.192454,0,25,2.079109,1
2,900.0,109.227791,153.000000,7,786,857,153.00,153.00,49.99,99.98,0.122477,0,24,2.091698,1
3,857.0,51.490000,128.884000,3,510,858,139.00,139.00,344.97,49.97,0.012164,0,4,6.960298,1
4,87.0,66.593200,171.474400,7,899,900,129.27,129.27,604.93,554.94,7.181852,0,3,5.149244,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499990,987.0,66.187500,250.000000,2,588,978,250.00,250.00,501.00,259.00,0.773773,0,3,0.308032,1
499993,588.0,121.913077,257.153846,2,620,588,250.00,268.60,544.96,644.95,1.050243,0,3,0.660340,1
499994,1020.0,187.490000,244.500000,6,899,899,244.50,244.50,864.96,569.96,0.008657,0,4,0.214685,1
499997,847.0,99.083636,80.200000,7,847,847,80.20,80.20,169.99,399.97,0.048738,0,3,0.047388,1


In [43]:
y = y[y['buy_post']==1]
y

,date_diff_post,buy_post
0,9.0,1
1,10.0,1
2,4.0,1
3,42.0,1
4,9.0,1
...,...,...
499990,3.0,1
499993,1.0,1
499994,55.0,1
499997,10.0,1


In [44]:
X = X.drop(columns=['buy_post'])
y = y.drop(columns=['buy_post'])

train_2 - файл готовый для обучения регрессии

In [56]:
X = X.drop(columns=['mean_delta_time'])

Работаем с date_diff_post

In [57]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression


model = RandomForestRegressor()

model.fit(X,y)

mean_squared_error(model.predict(X),y)

<ipython-input-57-29ac2696d641>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X,y)


27.85409057575293

In [58]:
r2_score(model.predict(X),y)

0.7849698471204264

In [60]:
df_test = pd.read_csv("ihateyou.csv")
df_test_f = pd.read_csv("main_test.csv")

In [61]:
df_test_f

,Unnamed: 0,index,customer_id,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval
0,0,0,13220760,666,40.141587,175.790476,8,607,178,174.40,74.10,2.00,419.98,-15.761007,0,11,-0.991379
1,1,1,30315975,789,142.490000,235.000000,1,789,845,235.00,235.00,584.96,144.98,-26.936829,0,3,0.000000
2,2,2,21679985,666,48.471053,209.526316,7,666,666,165.00,165.00,274.98,349.99,16.060093,0,8,5.267212
3,3,3,29754274,899,112.202632,98.421053,7,899,833,150.00,80.00,509.96,690.95,-21.969896,0,4,-5.492474
4,4,4,7797823,666,71.571529,114.500000,7,94,666,114.50,114.50,468.97,520.96,11.252500,0,12,2.748228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112329,112329,112329,14023374,469,104.990000,105.200000,7,469,803,105.20,105.20,99.99,44.99,-19.909722,0,3,-15.381258
112330,112330,112330,34923292,925,174.976667,241.733333,6,925,925,227.50,270.20,149.98,224.97,25.796794,0,3,-0.978383
112331,112331,112331,32761835,898,146.747500,123.640000,7,898,458,115.46,109.50,99.00,199.00,4.847801,0,4,1.239201
112332,112332,112332,38322785,478,233.656667,117.550000,8,478,956,117.55,117.55,599.98,99.99,0.013113,0,3,0.001331


In [62]:
df_test_f = df_test_f.drop(columns=['Unnamed: 0', 'index','mean_delta_time',])

In [65]:
df_test_f = df_test_f.drop(columns = ['customer_id'])

In [67]:
otevt = model.predict(df_test_f)

In [69]:
otevt

array([28.98, 28.44, 21.18, ..., 31.55, 33.13, 38.8 ])

In [72]:
otevt = pd.DataFrame(otevt)

In [73]:
otevt.to_csv("Otvet.csv")

In [50]:
import pickle

# save
with open('model_final.pkl','wb') as f:
    pickle.dump(model,f)
